In [16]:


# Python code demonstrate creating 
# DataFrame from dict narray / lists 
# By default addresses. 

import pandas as pd 
import numpy as np
import os.path

# initialize data of lists. 
data = {'Name': ['Tom 1', 'nick 1', 'krish 1', 'jack 1','Tom 2', 'nick 2', 'krish 2', 'jack 2'], 
		'Age': [20, 21, 19, 18, 20, 21, 19, 18]}

# Create DataFrame 
df = pd.DataFrame(data) 

mylist = [[i+1]*2 for i in range(len(df))]
df['group_id'] = np.resize(mylist,len(df))

# Print the output. 
print(df )


for i in range(4):
    print(i)
    df_new = df[df.group_id == i+1]
    print(df_new)
    if os.path.isfile('./prueba.csv') :
        df_new.to_csv('prueba_descarga.csv', mode='a', index=False, header=False)
    else:
        df_new.to_csv('prueba_descarga.csv')
        


      Name  Age  group_id
0    Tom 1   20         1
1   nick 1   21         1
2  krish 1   19         2
3   jack 1   18         2
4    Tom 2   20         3
5   nick 2   21         3
6  krish 2   19         4
7   jack 2   18         4
0
     Name  Age  group_id
0   Tom 1   20         1
1  nick 1   21         1
1
      Name  Age  group_id
2  krish 1   19         2
3   jack 1   18         2
2
     Name  Age  group_id
4   Tom 2   20         3
5  nick 2   21         3
3
      Name  Age  group_id
6  krish 2   19         4
7   jack 2   18         4


In [20]:
# Promedios mensuales de los ultimos 3 años

# pip install earthengine-api

import ee
import geopandas as gpd
import json
import pandas as pd
import time

start_time = time.perf_counter()

# https://developers.google.com/earth-engine/guides/service_account#use-a-service-account-with-a-private-key

service_account = 'gee-ine@ee-ricpizarrosine.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)


In [21]:
import geopandas as gpd

shapefile = gpd.read_file("gran_santiago/gran_santiago.shp")

# shapefile

In [22]:

### shapes in local to python
#shapefile = gpd.read_file("ips_shapes/ips_shapes.shp")
shapefile = gpd.read_file("gran_santiago/gran_santiago.shp")

shapefile = shapefile.iloc[0:10,]

features = []
for i in range(shapefile.shape[0]):
    geom = shapefile.iloc[i:i+1,:] 
    jsonDict = eval(geom.to_json()) 
    geojsonDict = jsonDict['features'][0] 
    features.append(ee.Feature(geojsonDict)) 

fc = ee.FeatureCollection(features)

print("crear feature")


crear feature


In [23]:

## NDVI function
def add_ndvi(image):
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['B8','B4']).rename("NDVI")
    image = image.addBands(ndvi)
    return(image)

## barrel soil function
formula_barrel_soil = "((b('B11') + b('B4')) - (b('B8') + b('B2'))) / ((b('B11') + b('B4')) + (b('B8') + b('B2')))"

def add_barrel_soil(image):
    barrel_soil =  image.expression(formula_barrel_soil).rename('BARREL_SOIL')
    image = image.addBands(barrel_soil)
    return(image)

## build index function
formula_build_index = "(b('B11') - b('B8')) / (b('B11') + b('B8'))";

def add_buil_index(image):
    build_index =  image.expression(formula_build_index).rename('BUILD_INDEX');
    return image.addBands(build_index)


# Import the Sentinel-2 image collection ##### fechas casem
S2_SR = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
          .filterBounds(fc)\
          .filterDate('2022-11-01', '2023-02-02')\
          .map(add_ndvi).map(add_barrel_soil).map(add_buil_index)

#print('Image collection:', S2_SR.first().getInfo())
 
# # Select the NDVI band and reduce all dates by mean
veget = S2_SR#.select('NDVI')
veget_red = veget.reduce(ee.Reducer.mean())

print("promedio tiempo")
#veget_red



promedio tiempo


In [26]:

### Calculate the mean NDVI inside each polygon
import geemap

def calcmean(feature):
      reduced = veget_red.reduceRegions(
                collection = feature,
                reducer = ee.Reducer.mean(),
                scale = 10)
    
      return reduced


final = fc.map(calcmean).flatten()

print("promedio regiones")
print(final.getInfo())
final

promedio regiones
{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-70.71599842799998, -33.41513349513221], [-70.71630188599994, -33.41501641613223], [-70.71668088699994, -33.41487903413223], [-70.71675386399994, -33.41485258113226], [-70.71688497799994, -33.41480505313221], [-70.71720983899996, -33.41465739213226], [-70.71737358099995, -33.41456350013225], [-70.71748112499995, -33.41449301713224], [-70.71761458899994, -33.41440554613224], [-70.71765070099997, -33.41438179113226], [-70.71766950599994, -33.41436942113224], [-70.71796070199997, -33.41417786913223], [-70.71801727499997, -33.41414270313224], [-70.71802757299997, -33.41413616713226], [-70.71803394599993, -33.41413212213224], [-70.71804416099997, -33.41412563913223], [-70.71815886299999, -33.41405284113222], [-70.71832958899995, -33.41394448613225], [-70.71843047499993, -33.41387996113223], [-70.71849407699995, -33.41383985713226], [-70.7186863849

In [27]:

#print(final.getInfo())

df = geemap.ee_to_pandas(final)


print("to df")

#print(df)

df.to_csv('more_indexs.csv')

end_time = time.perf_counter()

# Calculate elapsed time
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time/60)

to df
Elapsed time:  1.417245871666637
